# Client 동기화

In [5]:
import kfp
import requests

USERNAME = "user@example.com"
PASSWORD = "12341234"
NAMESPACE = "kubeflow-user-example-com"
HOST = "http://172.17.0.43:8080" # istio-ingressgateway pod ip:port

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": "user@example.com", "password": "12341234"}
session.post(response.url, headers=headers, data=data)
session_cookie = session.cookies.get_dict()["authservice_session"]

client = kfp.Client(
    host=f"{HOST}/pipeline",
    namespace=f"{NAMESPACE}",
    cookies=f"authservice_session={session_cookie}",
)
list_pipelines = client.list_pipelines()

for i in range(list_pipelines.total_size):
    print(list_pipelines.pipelines[i].name)
print(response.url)
print(session_cookie)

[Demo] XGBoost - Iterative model training
[Demo] TFX - Taxi tip prediction model trainer
[Tutorial] Data passing in python components
[Tutorial] DSL - Control structures
test
http://172.17.0.43:8080/dex/auth/local/login?back=&state=lz3bmbe26klpjm4ojdiw3cwox
MTY3MjU2MTQ4MnxOd3dBTkZCSFNFUllWemMwVkVKWVNWTlFVRE5NU0ZGSFVraEJVa0kwUXpaV1VsWmFRVWROVmtzeU1sVlVUVUpVVEVKRldFNURSMEU9fKARRzeW8z0wvE1Epj06MrlF_VAD-W1CN7kbJb4yG4p4


In [6]:
client

# Experiment 정보 가져오기

In [41]:
experiment_info = client.get_experiment(experiment_name="mnist-pipeline",namespace=NAMESPACE)

In [42]:
experiment_id = experiment_info.id

# Experiment 삭제하기

In [43]:
client.delete_experiment(experiment_id=experiment_id)

{}

# Runs 삭제하기

In [44]:
for i in range(client.list_runs().total_size):
    run_id = client.list_runs().runs[i].id
    client.runs.delete_run(id=run_id)
    print(run_id)

6d768f1f-e224-4bd0-87d0-4d871bbc8c85


## Pipeline Upload

In [45]:
pipeline_package_path='/home/jovyan/output_test.yaml'
pipeline_name='mnist-pipeline'
client.upload_pipeline(pipeline_package_path=pipeline_package_path,pipeline_name=pipeline_name)

{'created_at': datetime.datetime(2023, 1, 1, 9, 26, 28, tzinfo=tzlocal()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2023, 1, 1, 9, 26, 28, tzinfo=tzlocal()),
                     'id': '9eb7b704-8b2e-4696-ade3-54b12ea65538',
                     'name': 'mnist-pipeline',
                     'package_url': None,
                     'parameters': [{'name': 'no_epochs', 'value': None},
                                    {'name': 'optimizer', 'value': None}],
                     'resource_references': [{'key': {'id': '9eb7b704-8b2e-4696-ade3-54b12ea65538',
                                                      'type': 'PIPELINE'},
                                              'name': None,
                                              'relationship': 'OWNER'}]},
 'description': None,
 'error': None,
 'id': '9eb7b704-8b2e-4696-ade3-54b12ea65538',
 'name': 'mnist-pipeline',
 'parameters': [{'name': 'no_epochs', 'value': None},
    

# Experiment 생성

In [46]:
experiment = client.create_experiment(name=pipeline_name,description=pipeline_name,namespace=NAMESPACE)

## Run 생성

In [47]:
client.run_pipeline(experiment_id=experiment.id,job_name=pipeline_name,pipeline_package_path=pipeline_package_path)

{'created_at': datetime.datetime(2023, 1, 1, 9, 26, 31, tzinfo=tzlocal()),
 'description': None,
 'error': None,
 'finished_at': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
 'id': '775d2774-637e-4801-9e3f-4d78aeec0d40',
 'metrics': None,
 'name': 'mnist-pipeline',
 'pipeline_spec': {'parameters': None,
                   'pipeline_id': None,
                   'pipeline_manifest': None,
                   'pipeline_name': None,
                   'workflow_manifest': '{"apiVersion": '
                                        '"argoproj.io/v1alpha1", "kind": '
                                        '"Workflow", "metadata": '
                                        '{"generateName": '
                                        '"digits-recognizer-pipeline-", '
                                        '"annotations": '
                                        '{"pipelines.kubeflow.org/kfp_sdk_version": '
                                        '"1.6.3", '
                           